In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

LINK: https://colab.research.google.com/drive/1WV2oi2fh9XXyldh02wupFQX0wh5ZC-z-?usp=sharing

In [2]:
words = open('names.txt', 'r').read().splitlines()
len(words)

32033

In [3]:
chars = sorted(set(''.join(words)))
stoi = { s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0 
itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)

In [4]:
block_size = 3 # context length

def build_dataset(words):
    X, Y = [], [] 
    for w in words:
        context = [0] * block_size
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            context = context[1:] + [ix]
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(X.shape, Y.shape)
    return X,Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


In [5]:
# Utility function to compare manual gradients to Pytorch gradients
def cmp(s, dt, t):
    ex = torch.all(dt == t.grad).item()
    app = torch.allclose(dt, t.grad)
    maxdiff = (dt - t.grad).abs().max().item()
    print(f'{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}')

In [6]:
n_embd = 10 # dimensionality of the character embedding vectors
n_hidden = 64 # the number of neurons in the hidden layer of the MLP

g = torch.Generator().manual_seed(2147483647)  # for reproducibility
C = torch.randn((vocab_size, n_embd), generator=g) # Embedding matrix

# Layer 1:
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * (5/3)/((n_embd * block_size)**0.5)
b1 = torch.randn(n_hidden, generator=g) * 0.1 # using b1 just for fun, it is useless because of the batchnorm

# Layer 2:
W2 = torch.randn((n_hidden , vocab_size), generator=g) * 0.1
b2 = torch.randn(vocab_size, generator=g) * 0.1

# BatchNorm params:
bngain = torch.randn((1, n_hidden))*0.1 + 1.0
bnbias = torch.randn((1, n_hidden))*0.1

# Note: I am initializating many of these parameters in non-standard ways
# because sometimes initializating with e.g. all zeros could mask an incorrect
# implementation of the backward pass.
            
parameters = [C, W1, b1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
    p.requires_grad = True

4137


In [7]:
batch_size = 32
# construct a minibatch
ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
Yb, Xb = Ytr[ix], Xtr[ix]
Xb

tensor([[ 1,  1,  4],
        [18, 14,  1],
        [11,  5,  9],
        [ 0,  0,  1],
        [12, 15, 14],
        [ 0, 17,  1],
        [ 0,  0, 13],
        [13, 13,  1],
        [ 8, 25, 12],
        [ 0,  0, 26],
        [22, 15, 14],
        [19, 13,  9],
        [ 0,  0,  0],
        [ 0,  4,  5],
        [ 5, 14,  9],
        [18,  5,  5],
        [ 0,  4,  1],
        [ 1, 18,  1],
        [ 0,  0,  0],
        [ 0,  5, 12],
        [ 0, 10,  1],
        [ 9, 14,  1],
        [ 0,  0,  0],
        [ 0,  0, 18],
        [20,  5,  1],
        [ 0, 11, 15],
        [ 0,  0,  7],
        [ 0, 18,  5],
        [26,  5, 18],
        [ 0,  0, 14],
        [ 3,  5, 14],
        [ 0, 18, 15]])

In [8]:
# Forward pass: cut to smaller steps
emb = C[Xb] # Embed the characters into vectors
embcat = emb.view(emb.shape[0], -1) # Transform it to a 2D matrix from 3D

# Linear layer 1:
hprebn = embcat @ W1 + b1
print(hprebn.shape)

# BatchNorm layer (below there are small partitions of the mean and std to better represent during backprop):
bnmeani = 1/batch_size * hprebn.sum(0, keepdim=True) # hpreact.mean(0, keepdim=True) # batch mean
bndiff = hprebn - bnmeani
bndiff2 = bndiff**2
bnvar = 1/(batch_size-1)*(bndiff2).sum(0, keepdim=True) # note: Bessel's correction
bnvar_inv = (bnvar + 1e-5)**-0.5
bnraw = bndiff *bnvar_inv
hpreact = bngain * bnraw + bnbias

# Non-linearity layer:
h = torch.tanh(hpreact)

# Linear layer 2:
logits = h @ W2 + b2 # output layer

# Cross-entropy loss function (same as F.cross_entropy(logits, Yb)):
logit_maxes = logits.max(1, keepdim=True).values
norm_logits = logits - logit_maxes # subtract max for numerical stability
counts = norm_logits.exp()
counts_sum = counts.sum(1, keepdims=True)
counts_sum_inv = counts_sum**-1 # if I use (1.0 / counts_sum) instead then I can't get backprop to be bit exact...
probs = counts * counts_sum_inv
logprobs = probs.log()
loss = -logprobs[range(batch_size), Yb].mean()

# Pytorch backward pass
for p in parameters:
    p.grad = None
for t in [logprobs, probs, counts, counts_sum, counts_sum_inv, # afaik there is no cleaner way
          norm_logits, logit_maxes, logits, h, hpreact, bnraw,
         bnvar_inv, bnvar, bndiff2, bndiff, hprebn, bnmeani,
         embcat, emb]:
    t.retain_grad()

loss.backward()
loss

torch.Size([32, 64])


tensor(3.3382, grad_fn=<NegBackward0>)